# 📊 Projeto de Análise Exploratória de Dados (EDA)

## Introdução

Este projeto tem como objetivo realizar uma **Análise Exploratória de Dados (EDA)** utilizando informações de diferentes fontes relacionadas a canais de vendas e motoristas de entrega.  
A análise busca compreender melhor a estrutura dos dados, identificar padrões e gerar insights iniciais que possam apoiar decisões estratégicas.

## Objetivos do Projeto
- Explorar os dados disponíveis nos arquivos `channels.csv` e `drivers.csv`.
- Identificar a distribuição dos canais de vendas entre **OWN CHANNEL** e **MARKETPLACE**.
- Analisar o perfil dos motoristas, considerando **modal** (MOTOBOY ou BIKER) e **tipo de vínculo** (FREELANCE ou LOGISTIC OPERATOR).
- Realizar cruzamentos entre variáveis para entender possíveis relações.
- Preparar o ambiente para futuras análises mais completas, quando os arquivos maiores (`deliveries.csv`, `payments.csv`, `stores.csv`) puderem ser processados.

## Contexto
- **Channels**: representam os diferentes pontos de contato ou plataformas de vendas, classificados como canais próprios ou marketplaces.
- **Drivers**: representam os entregadores, com diferentes modais de transporte e tipos de vínculo contratual.

## Importância
A análise exploratória é uma etapa fundamental em projetos de ciência de dados, pois:
- Permite conhecer a qualidade e consistência dos dados.
- Facilita a identificação de tendências e padrões.
- Apoia a formulação de hipóteses para análises mais avançadas.
- Serve como base para modelos preditivos ou relatórios gerenciais.

---

Este notebook é, portanto, um **primeiro passo** para estruturar o entendimento dos dados disponíveis e abrir caminho para análises mais profundas envolvendo entregas, pagamentos e lojas.


Os dados foram extraídos do seguinte dataset https://www.kaggle.com/datasets/nosbielcs/brazilian-delivery-center

1. Informações e configurações

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import sys
print("Python:", sys.version)
print("pandas:", pd.__version__, "numpy:", np.__version__)
sns.set(style="whitegrid")


Python: 3.14.1 (tags/v3.14.1:57e0d17, Dec  2 2025, 14:05:07) [MSC v.1944 64 bit (AMD64)]
pandas: 2.3.3 numpy: 2.3.5
